In [2]:
!pip3 install beautifulsoup4 pandas numpy requests


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [3]:
from bs4 import BeautifulSoup
import requests as r
import pandas as pd
import numpy as np
import random

In [4]:
df = pd.read_csv("./data/title.akas.tsv", sep='\t')

/var/folders/sk/h402gsld7mx_v605wv7bblsc0000gn/T/ipykernel_82723/2445072090.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/title.akas.tsv", sep='\t')


In [5]:
df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [6]:
df.shape

(33893043, 8)

In [7]:
ids= df['titleId'].unique()

In [8]:
ids

array(['tt0000001', 'tt0000002', 'tt0000003', ..., 'tt9916850',
       'tt9916852', 'tt9916856'], dtype=object)

In [9]:
someids=random.choices(ids, k=5)


In [10]:
someids

['tt1276150', 'tt18337070', 'tt7073890', 'tt6962184', 'tt19049638']

In [11]:
base = "https://www.imdb.com/title/"

In [12]:
howls_ratings = BeautifulSoup(r.get("https://www.imdb.com/title/tt0347149/ratings/").text, 'html.parser')


In [13]:
type(howls_ratings)

bs4.BeautifulSoup

In [14]:
def ratings_url(url):
    return f"{base}{url}/ratings"

In [15]:
ts = howls_ratings.find_all("table")

In [16]:
type(ts[0])

bs4.element.Tag

In [18]:
rows = ts[0].find_all("tr")

In [19]:
def get_ratings_table(ratings_page):
    ratings_page = howls_ratings
    ts = ratings_page.find_all("table")
    rows = ts[0].find_all("tr")
    df = pd.DataFrame()
    scores = []
    nvotes = []
    for (i, row) in enumerate(rows):
        if i == 0:
            continue
        score = int(row.find_all("div", {"class": "rightAligned"})[0].text)
        nvote = int(row.find_all("div", {"class": "leftAligned"})[0].text.replace(",", ""))
        scores.append(score)
        nvotes.append(nvote)
        # df[i, :] = [score.text, nvotes.text]
    df = pd.DataFrame(list(zip(scores, nvotes)),columns =['score', 'nvotes'])
    return df

In [20]:
get_ratings_table(howls_ratings)

,score,nvotes
0,10,89226
1,9,95166
2,8,116340
3,7,58907
4,6,19000
5,5,6687
6,4,2750
7,3,1592
8,2,1163
9,1,3435


In [22]:
howls_id = "tt0347149"

In [23]:
url = f"{base}{howls_id}"

In [24]:
p = r.get(url).text

In [25]:
list(map(ratings_url, someids))

['https://www.imdb.com/title/tt1276150/ratings',
 'https://www.imdb.com/title/tt18337070/ratings',
 'https://www.imdb.com/title/tt7073890/ratings',
 'https://www.imdb.com/title/tt6962184/ratings',
 'https://www.imdb.com/title/tt19049638/ratings']